# OCEAN Data NFTs
Quickly create OCEAN Data NFTs by querying blockchain data from different providers.

Supported Blockchain Data Providers:
* [Dune](https://dune.com/)
* [Flipside](https://flipsidecrypto.xyz/)

Options:
1. Create Dune/Flipside queries and OCEAN Data NFTs from UI.
2. Pull Dune/Flipside data, store in file, and create OCEAN Data NFT from python.
    * Can pull from Flipside API
    * Dune query must already exist, unless have a "premium" subscription and can create Dune query using that
    * Honestly, if queries already exist, then should just use the Dune/Flipside hosted endpoints and don't need to store files yourself.
    * And honestly, if just using hosted endpoints, then just do from UI and reduce need for Infrura or risk of private keys leaking.
    * Store file in GitHub repo, must add code to commit to GitHub

Easiest User Steps:
* Fork repo
* Create new codespaces
* Add keys
* Create free Infrura account, if necessary
* Write Flipside query / Add Dune query
* 

In [ ]:
import json
import pandas as pd
import os
from dotenv import load_dotenv
import requests
from flipside import Flipside
from ocean_lib.web3_internal.utils import connect_to_network
from ocean_lib.example_config import get_config_dict
from ocean_lib.ocean.ocean import Ocean
from brownie.network import accounts

# Load variables from .env file
load_dotenv()

# Access environment variables
DUNE_API_KEY = os.getenv("DUNE_API_KEY")
FLIPSIDE_API_KEY = os.getenv("FLIPSIDE_API_KEY")

## Flipside

* API Docs: https://docs.flipsidecrypto.com/flipside-api/get-started
* Example Query: https://api.flipsidecrypto.com/api/v2/queries/f8e505dc-172b-44e5-99a6-8c35fe0695a5/data/latest

In [ ]:
# Query Flipside using their Python SDK
def query_flipside(sql):
    flipside = Flipside(FLIPSIDE_API_KEY, "https://api-v2.flipsidecrypto.xyz")
    results = flipside.query(sql)
    results_df = pd.DataFrame(results.rows, columns=results.columns)
    return results_df

In [ ]:
sql = """
SELECT
  date(block_timestamp) as dt,
  count(distinct tx_hash) as tx_ct
FROM ethereum.core.fact_transactions
WHERE block_timestamp >= GETDATE() - interval'7 days'
GROUP BY 1
order by 1 asc
"""
results_df = query_flipside(sql)
results_df

## Dune

* API Docs: https://dune.com/docs/api/
* Example Query: https://api.dune.com/api/v1/query/2296642/results?api_key=XXX

In [ ]:
# Query Dune Analytics using API
def query_dune(q):
    url = f"https://api.dune.com/api/v1/query/{q}/results?api_key={DUNE_API_KEY}"
    response = requests.get(url)
    results_json = json.loads(response.text)["result"]["rows"]
    results_df = pd.DataFrame.from_dict(results_json)
    return results_df

In [ ]:
results_df = query_dune(2296642)
results_df

## IPFS

Docs: https://docs.infura.io/networks/ipfs

## OCEAN Protocol

* Marketplace: https://market.oceanprotocol.com/
* SDK Docs: https://docs.oceanprotocol.com/developers/ocean.py/install

In [ ]:
# Create to Blockchain
connect_to_network("polygon-main")
config = get_config_dict("polygon-main")
ocean = Ocean(config)

In [ ]:
# Connect to Wallet
accounts.clear()
private_key = os.getenv('PRIVATE_KEY')
wallet = accounts.add(private_key)
print('MATIC Balance:', wallet.balance() / 1e18 )

In [ ]:
# Create Data NFT
name = "Data NFT Test 1"
url = "https://raw.githubusercontent.com/trentmc/branin/main/branin.arff"
(data_nft, datatoken, ddo) = ocean.assets.create_url_asset(name, url, {"from": wallet})

print("Data NFT published:")
print(f"  data_nft: symbol={data_nft.symbol()}, address={data_nft.address}")
print(f"  datatoken: symbol={datatoken.symbol()}, address={datatoken.address}")
print(f"  did={ddo.did}")